In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import math
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

import time

In [2]:
mod = SourceModule("""
    # include <stdio.h>
    
    #define N 1000
    
    
    __global__ void doIndexy(float *Z, int *indexx, int *indexy, int *P1, float *P3, int total )
    { 
        
        int idx = blockIdx.x * blockDim.x + threadIdx.x; 

        if(idx >= total){
            return;
        }
        //printf("%d\\n", total);
        float sum = 0;
        int j = indexy[idx]; 
        for (int i = 0; i < total ; i++){
           if(Z[indexx[i]+j-1]>0.f){
               sum += 0.1*0.1*0.1 * Z[indexx[i]+j-1] * P1[i];
           }
           
           
        }; 
        //printf("%d\t%f\t%f\t%d\t%d\t%d%d\\n",idx,sum,Z[195040],indexx[0], indexy[0],P1[0],total);
        P3[idx] = sum; 
        
        //printf ("%d:::::::\\t", idx);
       
    }""")

In [3]:
# calculate equation 1
def eq1(t2,t3,t4,T):
    a=np.empty(len(t4))
    for i in range(0,len(t4)):
        arr = np.array([t2[i],t3[i],t4[i],0])
        arr = np.sort(arr, axis=None)
        a[i]=1- np.heaviside (np.absolute(arr[3] -arr[1])- T, 0)
    return a

In [4]:
# for N=3 
def eq4(a,b,c,tau=1):
    ps = np.empty(len(a))
    for i in range(len(a)):
        if(b[i] >= a[i]) and (c[i]>=a[i]) and (c[i] >= b[i]):
            ps[i] = (1/tau**3)*(math.exp(-1*c[i]/tau))
        else: 
            ps[i] = 0
    return ps

In [ ]:
# parameter
tau=1;
T=1;
mu=1.0;
lamda=1.0;
bin_size=0.10;
Pmin=0;
Pmax=15;
tmin=-10;
tmax=Pmax - tmin;

In [ ]:
start = time.time()
Pin_point= int((Pmax-Pmin)/bin_size +1);
obs_point=int((tmax-tmin)/bin_size + 1);
Zmax=tmax;
Zmin=tmin-Pmax;
Z_point=int((Zmax-Zmin)/bin_size + 1);

t4, t3,t2 = np.meshgrid(np.arange(tmin,tmax+0.001,bin_size), np.arange(tmin,tmax+0.001,bin_size), np.arange(tmin,tmax+0.001,bin_size));
t2 = t2.ravel()
t3 = t3.ravel()
t4 = t4.ravel()

tau4,tau3,tau2 = np.meshgrid(np.arange(Pmin,Pmax+0.001,bin_size), np.arange(Pmin,Pmax+0.001,bin_size), np.arange(Pmin,Pmax+0.001,bin_size));
tau2 = tau2.ravel()
tau3 = tau3.ravel()
tau4 = tau4.ravel()

p1=eq1(t3,t4,t2,T)

f2 = pd.read_csv('savedist_4d.tsv',sep=' ',squeeze=True,header=None).values

# for 4
indexy = np.array([],dtype=int)
for k in range (1,Pin_point+1):
    for k1 in range (1,Pin_point+1):
        a = [i for i in range(1+(Z_point*Z_point*(k-1))+(Z_point*(k1-1)),Pin_point+1+(Z_point*Z_point*(k-1))+(Z_point*(k1-1)))] 
        indexy =np.append(indexy,a)
        
indexx = np.array([],dtype=int)
for k in range (0,obs_point): 
    for k1 in range (0,obs_point): 
        a = [i for i in range(Z_point*Z_point*(k)+k1*(Z_point),Z_point*Z_point*(k)+k1*Z_point+obs_point)] 
        indexx =np.append(indexx,a)

# p3=np.zeros(len(tau2));
# for i in range(0,len(tau2)):
#     p2=f2[indexx+indexy[i]-1]
#     p3[i]=bin_size*bin_size*bin_size*np.sum(np.multiply(p1,p2));

p4 = eq4(tau3,tau4,tau2,tau)

# p = bin_size*bin_size*bin_size*np.sum(np.multiply(p4,p3))
# print("Prob: ", p, "Sec: ", time.time() - start)

In [ ]:
tau2

In [ ]:
tau3[0:200]

In [ ]:
tau4[0:200]

In [ ]:
indexx.shape

In [ ]:
indexy.shape

In [ ]:
startC=time.time()
d_Z = cuda.mem_alloc(np.float32(f2).nbytes)
cuda.memcpy_htod(d_Z, np.float32(f2))

d_indexx = cuda.mem_alloc(np.int32(indexx).nbytes)
cuda.memcpy_htod(d_indexx, np.int32(indexx))

d_indexy = cuda.mem_alloc(np.int32(indexy).nbytes)
cuda.memcpy_htod(d_indexy, np.int32(indexy))

d_P1 = cuda.mem_alloc(np.int32(p1).nbytes) 
cuda.memcpy_htod(d_P1, np.int32(p1))

d_P3 = cuda.mem_alloc(np.float32(indexy).nbytes) 

In [ ]:
func = mod.get_function("doIndexy")

blocksize = 128
gridsize = math.floor(len(indexy)/blocksize)+1
func(d_Z, d_indexx, d_indexy, d_P1, d_P3, np.int32(len(p1)),  block=(blocksize,1,1), grid =(gridsize,1,1))

h_test_out = np.zeros(len(indexy), np.float32)
cuda.memcpy_dtoh(h_test_out, d_P3)

cuda.Context.synchronize()

p = bin_size*bin_size*bin_size*np.sum(np.multiply( p4, h_test_out))
print("Prob: ", p, "Sec: ", time.time() - start, "inCuda ", time.time()- startC)
